PWD's Wingohocking SWMM Model - runs to test effects of GSI installations

In [10]:
# Run this block to load imports,
# to test mongodb connectivity, and view existing collections
%reset
import yaml
from swmm_objects import *
from swmm_read_cso_time_series import *
from swmm_run_cso import *
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
client = MongoClient(mongoServer)
database = "ewri2017"
username = "arthur"
password = "jXlIkLfu5Dfr"
authenticationDatabase = "admin"
db = client[database]
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionNames = sorted(db.collection_names())
collectionNames

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


[u'july', u'july2008_1']

In [15]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking
# model BEFORE adding GSI to obtain the zero LID runoff and CSO values

#  IMPORTANT:  Set these for each run:
#################################################
collection = '2008annual_1'
swmmInpFile = "wingohocking_PWD.inp"  # initial SWMM .inp file
lidZeroFile = "PWD_wingo_Wakefield_list_zero.yaml"
################################################

db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database

thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection)  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

0.0 min, 16.50 sec
volume = 183.249
Stored record 5835ff775219aa2b49721d5f for zero LID usage:  peak = 1906.45, volume = 183.249


### Loop ALL subcatchment LID numbers over a range:

In [16]:
# Now, apply a range of Wakefield Park rain gardens in all seven subcatchments simultaneously
from copy import deepcopy
for numberForAllLid in range(1,10):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"

0.0 min, 16.43 sec
volume = 183.137
0.0 min, 16.62 sec
volume = 183.026
0.0 min, 16.48 sec
volume = 182.915
0.0 min, 16.46 sec
volume = 182.804
0.0 min, 16.74 sec
volume = 182.692
0.0 min, 16.46 sec
volume = 182.581
0.0 min, 17.28 sec
volume = 182.47
0.0 min, 16.90 sec
volume = 182.359
0.0 min, 16.81 sec
volume = 182.247
FINISHED ALL RUNS


### To display all of the runs from the database:

In [3]:
runs = db[collection]
cursor = runs.find()
#\for run in cursor:
#    print run
#    print ''

### To Reconstitute the SWMM input file for any of the runs above

In [4]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
#print swmmInputFileStr
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()